# Open DataTree with `engine=` parameter

This notebook demonstrates the new unified `open_datatree` API that allows opening radar files as `xarray.DataTree` using the `engine=` parameter.

Three ways to open a DataTree:
- `xd.open_datatree(file, engine="...")` — xradar unified API
- `xr.open_datatree(file, engine="...")` — xarray native API
- `xd.io.open_*_datatree(file)` — legacy per-format functions (deprecated, emit `FutureWarning`)

In [ ]:
import warnings

import xarray as xr
from open_radar_data import DATASETS

import xradar as xd

## Download test data

Fetching radar data files from [open-radar-data](https://github.com/openradar/open-radar-data) repository.

In [ ]:
odim_file = DATASETS.fetch("71_20181220_060628.pvol.h5")
cfradial1_file = DATASETS.fetch("cfrad.20080604_002217_000_SPOL_v36_SUR.nc")
nexrad_file = DATASETS.fetch("KATX20130717_195021_V06")

## 1. `xd.open_datatree()` — Unified xradar API

The new unified entry point. Specify the `engine` to select the backend.

### ODIM_H5

In [ ]:
dtree = xd.open_datatree(odim_file, engine="odim")
display(dtree)

The tree follows the CfRadial2 group structure with metadata groups at the root level and sweep groups below.

In [ ]:
# Root dataset contains global metadata
display(dtree.ds)

In [ ]:
# Access a specific sweep
display(dtree["sweep_0"].ds)

In [ ]:
# Metadata groups
print("radar_parameters:", list(dtree["radar_parameters"].ds.data_vars))
print(
    "georeferencing_correction:", list(dtree["georeferencing_correction"].ds.data_vars)
)
print("radar_calibration:", list(dtree["radar_calibration"].ds.data_vars))

### CfRadial1

In [ ]:
dtree = xd.open_datatree(cfradial1_file, engine="cfradial1")
display(dtree)

In [ ]:
dtree["sweep_0"].ds.DBZ.plot()

### NEXRAD Level 2

In [ ]:
dtree = xd.open_datatree(nexrad_file, engine="nexradlevel2")
display(dtree)

In [ ]:
dtree["sweep_0"].ds.DBZH.plot()

## 2. Sweep selection

Select specific sweeps by index (int or list) or by name.

In [ ]:
# Single sweep by index
dtree = xd.open_datatree(odim_file, engine="odim", sweep=0)
print("Children:", list(dtree.children))

In [ ]:
# Multiple sweeps by index
dtree = xd.open_datatree(odim_file, engine="odim", sweep=[0, 2, 4])
print("Children:", list(dtree.children))

In [ ]:
# Sweeps by name
dtree = xd.open_datatree(
    cfradial1_file, engine="cfradial1", sweep=["sweep_0", "sweep_3"]
)
print("Children:", list(dtree.children))

## 3. Backend kwargs

Pass backend-specific options directly as keyword arguments.

In [ ]:
# first_dim controls the leading dimension ("auto" uses azimuth/elevation)
# site_coords attaches latitude/longitude/altitude to sweep datasets
dtree = xd.open_datatree(
    odim_file,
    engine="odim",
    sweep=[0],
    first_dim="auto",
    site_coords=True,
)
sweep_ds = dtree["sweep_0"].ds
print("Dimensions:", dict(sweep_ds.dims))
print("Site coords present:", "latitude" in sweep_ds.coords)

## 4. `xr.open_datatree()` — xarray native API

The same backends work directly with xarray's native `open_datatree`, no xradar wrapper needed.

In [ ]:
dtree = xr.open_datatree(odim_file, engine="odim")
display(dtree)

In [ ]:
dtree = xr.open_datatree(nexrad_file, engine="nexradlevel2", sweep=[0, 1])
display(dtree)

## 5. `open_groups_as_dict()` — Low-level access

For advanced use, get the raw `dict[str, Dataset]` before it becomes a DataTree.

In [ ]:
from xradar.io.backends.odim import OdimBackendEntrypoint

backend = OdimBackendEntrypoint()
groups = backend.open_groups_as_dict(odim_file, sweep=[0, 1])

print("Group keys:", list(groups.keys()))
print()
print("Root dataset:")
display(groups["/"])

## 6. Backward compatibility — deprecated functions

The legacy per-format functions still work but emit a `FutureWarning` directing you to the new API.

In [ ]:
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    dtree_old = xd.io.open_odim_datatree(odim_file, sweep=[0])
    for warning in w:
        if issubclass(warning.category, FutureWarning):
            print(f"FutureWarning: {warning.message}")

display(dtree_old)

In [ ]:
# The old and new APIs produce equivalent results
dtree_new = xd.open_datatree(odim_file, engine="odim", sweep=[0])
print("Same children:", set(dtree_old.children) == set(dtree_new.children))

## 7. Error handling

In [ ]:
# Unknown engine raises a clear error
try:
    xd.open_datatree(odim_file, engine="nonexistent")
except ValueError as e:
    print(f"ValueError: {e}")

## Summary

| API | Example | Status |
|-----|---------|--------|
| `xd.open_datatree(file, engine="odim")` | Unified xradar API | **New** |
| `xr.open_datatree(file, engine="odim")` | xarray native API | **New** |
| `xd.io.open_odim_datatree(file)` | Per-format function | Deprecated |

Supported engines: `"odim"`, `"cfradial1"`, `"nexradlevel2"`